In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Get hit type

In [2]:
import json

with open('/content/drive/MyDrive/Badminton-AI/badminton-ai-core/draft/HitDetect/hit_types.json', 'r') as file:
    hit_types = json.load(file)

hit_types

['clear',
 'defensive shot',
 'drive',
 'drop',
 'lob',
 'long service',
 'net shot',
 'push/rush',
 'short service',
 'smash']

In [3]:
def idx_to_onehot(idx, num_classes):
    """
    Converts an index to a one-hot encoded vector with the first position reserved for null.

    Parameters:
    - idx: The index to convert, with 0 being the first actual category.
    - num_classes: The total number of categories excluding the null category.

    Returns:
    - A one-hot encoded vector with size (num_classes) to include the null category.
    """
    # Initialize a vector of zeros with length num_classes (for the null category)
    onehot = [0]*(num_classes)
    # Increment the idx by 1 to reserve the first position for null and set the appropriate position to 1
    onehot[idx] = 1
    return onehot

In [4]:

import torch
from torch.utils.data import Dataset
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import pandas as pd
import torch
from torch.utils.data import Dataset
import numpy as np
from torch.utils.data import DataLoader
import os
import random

'''
human keypoints: 17*2
court keypoints: 6*2
net keypoints:   4*2
ball keypoints:  1*2
total keypoints: 28*2

'''




class HitTypeDataset(Dataset):
    def __init__(self, dataset_folder,num_consecutive_frames,normalization=True):
        self.dataset=[]
        self.positive=0
        self.negative=0
        # 遍历文件夹及其子文件夹，找到所有的CSV文件路径
        cnt=0
        for root, dirs, files in os.walk(dataset_folder):
            for file in files:
                if file.endswith(".csv"):
                    # 定义处理函数
                    data_path=os.path.join(root, file)
                    print(data_path)

                    try:
                      df= pd.read_csv(data_path, converters={"ball": eval,"top":eval,"bottom":eval,"court":eval,"net":eval})
                    except:
                      print('Error! cannot process: ', data_path)
                      continue

                    rows = len(df)
                    remainder = rows % 12
                    if remainder > 0:
                        num_to_pad = 12 - remainder
                    else:
                        num_to_pad = 0

                    if num_to_pad > 0:
                        last_row = df.iloc[-1]
                        padding_data = np.tile(last_row.values, (num_to_pad, 1))
                        padded_df = pd.DataFrame(padding_data, columns=df.columns)
                        df = pd.concat([df, padded_df], axis=0)
                        df = df.reset_index(drop=True)

                    small_dataset =df

                    for i in range(len(small_dataset)):
                        if i%num_consecutive_frames!=0:
                            continue
                        if i>=len(small_dataset)-num_consecutive_frames:
                            break
                        oridata=small_dataset.loc[i:i+num_consecutive_frames-1,:].copy()
                        oridata=oridata.reset_index(drop=True)
                        data=[]

                        target1=None

                        for index,row in oridata.iterrows():
                            pos=np.array(row['type'])
                            if str(pos) in hit_types:
                                if target1 is None:
                                    target1=idx_to_onehot(hit_types.index(pos), len(hit_types))
                                    self.positive+=1

                            pos=np.array(row['pos'])
                            if str(pos)=='top':
                                who_hit = [1, 0]
                            elif str(pos)=='bottom':
                                who_hit = [0, 1]
                            else:
                                who_hit = [0, 0]

                            top=np.array(row['top']).reshape(-1,2)
                            bottom=np.array(row['bottom']).reshape(-1,2)
                            court=np.array(row['court']).reshape(-1,2)
                            net=np.array(row['net']).reshape(-1,2)
                            ball=np.array(row['ball']).reshape(-1,2)
                            who_hit=np.array(who_hit).reshape(-1,2)

                            frame_data = np.concatenate((top, bottom, court, net, ball, who_hit), axis=0)

                            if normalization:
                                frame_data[:,0]/=1920
                                frame_data[:,1]/=1080
                            data.append(frame_data.reshape(1,-1))
                        data=np.array(data)
                        if target1 is None:
                            continue
                            # if self.negative>self.positive:
                            #     continue
                            # target1=[0] * (len(hit_types) + 1)
                            # self.negative+=1
                        self.dataset.append((data.reshape(-1),target1))


    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        # 假设每个样本是一个元组 (input, target)
        sample = self.dataset[index]
        input_data = sample[0]
        target1 = sample[1]

        # 转换为Tensor对象
        input_tensor = torch.tensor(input_data)
        target1_tensor = torch.tensor(target1)

        return input_tensor, target1_tensor


num_consecutive_frames=30
batch_size=30
shuffle=True
normalization=True

TrainDataset=HitTypeDataset("/content/drive/MyDrive/Badminton-AI/badminton-ai-core/draft/HitDetect/train",num_consecutive_frames,normalization)
ValidDataset=HitTypeDataset("/content/drive/MyDrive/Badminton-AI/badminton-ai-core/draft/HitDetect/valid",num_consecutive_frames,normalization)

/content/drive/MyDrive/Badminton-AI/badminton-ai-core/draft/HitDetect/train/Akane_YAMAGUCHI_AN_Se_Young_DAIHATSU_YONEX_Japan_Open_2022_Finals/rally_1-15.csv
/content/drive/MyDrive/Badminton-AI/badminton-ai-core/draft/HitDetect/train/Akane_YAMAGUCHI_AN_Se_Young_DAIHATSU_YONEX_Japan_Open_2022_Finals/rally_1-24.csv
/content/drive/MyDrive/Badminton-AI/badminton-ai-core/draft/HitDetect/train/Akane_YAMAGUCHI_AN_Se_Young_DAIHATSU_YONEX_Japan_Open_2022_Finals/rally_1-10.csv
/content/drive/MyDrive/Badminton-AI/badminton-ai-core/draft/HitDetect/train/Akane_YAMAGUCHI_AN_Se_Young_DAIHATSU_YONEX_Japan_Open_2022_Finals/rally_2-10.csv
/content/drive/MyDrive/Badminton-AI/badminton-ai-core/draft/HitDetect/train/Akane_YAMAGUCHI_AN_Se_Young_DAIHATSU_YONEX_Japan_Open_2022_Finals/rally_1-21.csv
/content/drive/MyDrive/Badminton-AI/badminton-ai-core/draft/HitDetect/train/Akane_YAMAGUCHI_AN_Se_Young_DAIHATSU_YONEX_Japan_Open_2022_Finals/rally_1-6.csv
/content/drive/MyDrive/Badminton-AI/badminton-ai-core/draft

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ShotTypeModel(nn.Module):
    def __init__(self, feature_dim, num_consecutive_frames, num_classes):
        super(ShotTypeModel, self).__init__()
        self.num_consecutive_frames = num_consecutive_frames
        self.feature_dim = feature_dim

        self.gru1 = nn.GRU(feature_dim // num_consecutive_frames, 64, bidirectional=True, batch_first=True)
        self.gru2 = nn.GRU(128, 64, bidirectional=True, batch_first=True)
        self.global_maxpool = nn.MaxPool1d(num_consecutive_frames)
        self.dense = nn.Linear(128, num_classes)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        batch_size = x.shape[0]
        x=x.float()
        x = x.view(batch_size, self.num_consecutive_frames, self.feature_dim // self.num_consecutive_frames)
        x, _ = self.gru1(x)
        x, _ = self.gru2(x)
        x = x.transpose(1, 2)
        x = self.global_maxpool(x).squeeze()
        x= self.dense(x)
        x=self.softmax(x)
        return x



print(TrainDataset.positive,TrainDataset.negative)
print(ValidDataset.positive,ValidDataset.negative)

feature_dim=92*num_consecutive_frames
train_data_loader = DataLoader(TrainDataset, batch_size=batch_size, shuffle=shuffle)

valid_data_loader = DataLoader(ValidDataset, batch_size=batch_size, shuffle=shuffle)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=ShotTypeModel(feature_dim,num_consecutive_frames, len(hit_types))
criterion = nn.CrossEntropyLoss()
model.to(device)
criterion.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 100



train_loss_list = []
valid_loss_list=[]
for epoch in range(num_epochs):
    train_loss_sum=0
    model.train()
    for batch_data in train_data_loader:
        inputs, labels = batch_data
        inputs = inputs.to(device)  # 将输入数据移动到设备上
        labels = labels.to(device).float()  # 将输入数据移动到设备上

        outputs = model(inputs)
        outputs=outputs.reshape(-1, len(hit_types))

        train_loss = criterion(outputs,labels)
        train_loss_sum+=train_loss.detach()
        # 反向传播和优化
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        # 执行你的训练或测试操作
    train_loss_list.append(train_loss_sum)


    # 打印训练信息
    if (epoch + 1) % 1== 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch + 1, num_epochs,
                                                    train_loss.item()))
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in valid_data_loader:
            inputs = inputs.to(device)  # 将输入数据移动到设备上
            labels=labels.to(device)
            # 前向传播
            outputs = model(inputs)
            outputs=outputs.reshape(-1, len(hit_types))


            y_true=torch.argmax(labels,axis=1)
            y_pred=torch.argmax(outputs,axis=1)
            total+=len(y_true)
            correct+=(y_true==y_pred).sum().item()
        if total==0:
            print(f'Accuracy on test set: {0}')
            continue



        print(f'Accuracy on test set: {correct/total}')

13962 0
894 0
Epoch [1/100], Loss: 2.0216
Accuracy on test set: 0.48322147651006714
Epoch [2/100], Loss: 2.0409
Accuracy on test set: 0.5134228187919463
Epoch [3/100], Loss: 1.9606
Accuracy on test set: 0.3814317673378076
Epoch [4/100], Loss: 1.9216
Accuracy on test set: 0.5525727069351231
Epoch [5/100], Loss: 1.9348
Accuracy on test set: 0.5637583892617449
Epoch [6/100], Loss: 2.1223
Accuracy on test set: 0.5961968680089486
Epoch [7/100], Loss: 1.8798
Accuracy on test set: 0.5961968680089486
Epoch [8/100], Loss: 1.7996
Accuracy on test set: 0.6252796420581656
Epoch [9/100], Loss: 1.9864
Accuracy on test set: 0.5827740492170023
Epoch [10/100], Loss: 1.7164
Accuracy on test set: 0.6263982102908278
Epoch [11/100], Loss: 1.5759
Accuracy on test set: 0.6196868008948546
Epoch [12/100], Loss: 1.9494
Accuracy on test set: 0.5917225950782998
Epoch [13/100], Loss: 1.8775
Accuracy on test set: 0.6331096196868009
Epoch [14/100], Loss: 2.0628
Accuracy on test set: 0.6118568232662193
Epoch [15/100]

In [6]:
# 保存整个模型
torch.save(model, '/content/drive/MyDrive/Badminton-AI/badminton-ai-core/draft/HitDetect/shot_type_detect.pth')

# print(f"best_accuracy={best_accuracy}")
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in valid_data_loader:
        inputs = inputs.to(device)  # 将输入数据移动到设备上
        labels = labels.to(device).float()  # 将标签移动到设备上

        # 前向传播
        outputs = model(inputs)

        y_true=torch.argmax(labels,axis=1)
        y_pred=torch.argmax(outputs,axis=1)
        print(y_true)
        print(y_pred)
        total+=len(y_true)
        correct+=(y_true==y_pred).sum().item()


    print(f'Accuracy on test set: {correct/total}')




tensor([4, 8, 4, 4, 1, 3, 8, 1, 6, 1, 7, 2, 4, 0, 6, 6, 6, 0, 0, 1, 9, 9, 6, 2,
        4, 6, 9, 4, 9, 6], device='cuda:0')
tensor([4, 8, 4, 4, 1, 3, 8, 1, 6, 4, 4, 4, 4, 0, 6, 6, 6, 0, 0, 6, 9, 9, 6, 6,
        4, 6, 3, 4, 9, 6], device='cuda:0')
tensor([7, 4, 1, 6, 3, 3, 6, 4, 4, 8, 0, 3, 4, 9, 9, 5, 0, 9, 8, 8, 3, 0, 0, 9,
        6, 7, 0, 1, 9, 8], device='cuda:0')
tensor([6, 4, 3, 3, 9, 9, 6, 4, 4, 8, 0, 0, 4, 9, 9, 4, 0, 9, 8, 8, 9, 0, 0, 3,
        6, 6, 0, 1, 3, 8], device='cuda:0')
tensor([4, 6, 2, 7, 6, 6, 2, 6, 3, 1, 1, 0, 9, 9, 0, 1, 7, 6, 9, 0, 1, 4, 4, 4,
        0, 7, 6, 9, 6, 6], device='cuda:0')
tensor([4, 6, 6, 4, 1, 6, 6, 6, 9, 1, 1, 0, 9, 9, 0, 4, 4, 6, 9, 0, 1, 4, 6, 4,
        0, 7, 6, 9, 6, 6], device='cuda:0')
tensor([6, 6, 8, 9, 3, 6, 9, 0, 0, 0, 7, 4, 7, 8, 0, 6, 7, 9, 6, 4, 3, 0, 7, 9,
        9, 8, 0, 9, 9, 4], device='cuda:0')
tensor([6, 6, 8, 9, 6, 6, 9, 0, 0, 0, 4, 4, 6, 8, 0, 6, 7, 9, 6, 4, 3, 0, 7, 9,
        3, 8, 0, 9, 3, 4], device='cuda:0')
tensor([

In [ ]:
final_list=[]
true_list=[]
data_path="/content/drive/MyDrive/Badminton_Player_Analysis_Project/SoloShuttlePose/draft/HitDataset/valid/Viktor_AXELSEN_Jonatan_CHRISTIE_Malaysia_Open_2022_SemiFinals/rally_1-12.csv"
df= pd.read_csv(data_path, converters={"ball": eval,"top":eval,"bottom":eval,"court":eval,"net":eval})

rows = len(df)
remainder = rows % 12
if remainder > 0:
    num_to_pad = 12 - remainder
else:
    num_to_pad = 0

if num_to_pad > 0:
    last_row = df.iloc[-1]
    padding_data = np.tile(last_row.values, (num_to_pad, 1))
    padded_df = pd.DataFrame(padding_data, columns=df.columns)
    df = pd.concat([df, padded_df], axis=0)
    df = df.reset_index(drop=True)

small_dataset =df
probs_list=[]
for i in range(len(small_dataset)):

    if i>=len(small_dataset)-num_consecutive_frames:
        break


    oridata=small_dataset.loc[i:i+num_consecutive_frames-1,:].copy()
    oridata=oridata.reset_index(drop=True)
    data=[]
    # print(oridata)
    if str(oridata.loc[0,'pos'])=='nan':
            true_list.append(0)
    elif str(oridata.loc[0,'pos'])=='top':
            true_list.append(1)
    elif str(oridata.loc[0,'pos'])=='bottom':
            true_list.append(2)

    for index,row in oridata.iterrows():

        top=np.array(row['top']).reshape(-1,2)
        bottom=np.array(row['bottom']).reshape(-1,2)
        court=np.array(row['court']).reshape(-1,2)
        # net=np.array(row['net']).reshape(-1,2)
        ball=np.array(row['ball']).reshape(-1,2)

        frame_data = np.concatenate((top, bottom, court, ball), axis=0)

        if normalization:
            # 将 x 坐标从 0 到 1920 映射到 1 到 2
            frame_data[:,0] /=1920

            # 将 y 坐标从 0 到 1080 映射到 1 到 2
            frame_data[:,1] /=1080

        data.append(frame_data.reshape(1,-1))
    data=np.array(data).reshape(1,-1)
    # print(data.shape)
    # if i%num_consecutive_frames!=0:
    #     final_list.append(0)
    #     continue
    outputs=model(torch.FloatTensor(data).to(device))

    # print(outputs)
    pred=torch.argmax(outputs).item()
    probs=outputs[pred].item()
    final_list.append(pred)

print(true_list)
print(final_list)


[2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1]
[0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,